In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import random

In [ ]:
def MaxMinDiversity(subset: np.ndarray, complete: np.ndarray) -> float:
    """
    Calculate Max-Min Diversity, the minimum distance between all pairs of points in the given subset.
    
    Parameters:
        subset: Subset of points
        complete: Complete graph adjacency matrix containing distances between all pairs of points
    
    Returns:
        min_dist: Max-Min Diversity, the minimum distance between all pairs of points in the subset
    """
    if len(subset) < 2:
        return 0
    
    n = len(subset)
    min_dist = float('inf')
    
    for i in range(n):
        for j in range(i + 1, n):
            dist = complete[subset[i]][subset[j]]
            min_dist = min(min_dist, dist)
    
    return min_dist

In [ ]:
def distancePS(centerSet: np.ndarray, i: int, complete: np.ndarray) -> float:
    """
    Returns the distance between a certain point and a certain set.
    
    Parameters:
        centerSet: A numpy array containing confirmed center indexes
        i: The index of any point
        complete : Complete graph adjacency matrix containing distances between all pairs of points
    
    Returns:
        min_distance: The distance between point and center set
    """
    min_distance = float("inf")
    for center in centerSet:
        distance = complete[center][i]
        if (distance < min_distance):
            min_distance = distance
    
    return min_distance

In [ ]:
def GMM(points_index: np.ndarray, k: int, complete: np.ndarray) -> np.ndarray:
    """
    Returns indexes of k centers after running GMM Algorithm.
    
    Parameters: 
        points_index: The indexes of data
        k: A decimal integer, the number of centers
        complete: Complete graph adjacency matrix containing distances between all pairs of points
        initial: An initial set of elements
    
    Returns:
        centers: A numpy array with k indexes as center point indexes
    """
    centers = []
    initial_point_index = random.choice(points_index)
    centers.append(initial_point_index)
    while (len(centers) < k):
        max_distance = 0
        max_distance_vector_index = None
        for i in points_index:
            distance = distancePS(centers, i, complete)
            if distance > max_distance:
                max_distance = distance
                max_distance_vector_index = i
        centers.append(max_distance_vector_index)
    centers = np.array(centers)

    return centers

In [ ]:
def ILP(n, E, k, color_constraints, color_sets):

    index_mapping = dict()
    new_n = 0
    for c, nodes in color_sets.items():
        for node in nodes:
            if node not in index_mapping:
                index_mapping[node] = new_n
                new_n += 1
    
    if len(index_mapping) != n:
        raise ValueError("There must be something wrong in the FMMD.")

    model = gp.Model("ILP")

    x = model.addVars(n, vtype=GRB.BINARY, name="x")

    model.setObjective(gp.quicksum(x[i] for i in range(n)), GRB.MAXIMIZE)

    for (i, j) in E:
        if i in index_mapping and j in index_mapping:
            new_i = index_mapping[i]
            new_j = index_mapping[j]
            model.addConstr(x[new_i] + x[new_j] <= 1, f"edge_{new_i}_{new_j}")

    model.addConstr(gp.quicksum(x[i] for i in range(n)) <= k, "max_selection")

    for c, number in color_constraints.items():
        nodes_in_color = color_sets[c]
        model.addConstr(gp.quicksum(x[index_mapping[i]] for i in nodes_in_color if i in index_mapping) == number, f"number_color_{c}")

    model.optimize()

    if model.status == GRB.OPTIMAL:
        selected_nodes = np.array([i for i in range(n) if x[i].x > 0.5])
        # 将选中的节点从新索引映射回原始节点的索引
        selected_original_nodes = np.array([node for new_index in selected_nodes for node, idx in index_mapping.items() if idx == new_index])
        return selected_original_nodes
    else:
        raise ValueError("No optimal solution found.")
        

In [ ]:
def FMMDS(sets: tuple, k: int, error: float, complete: np.ndarray) -> np.ndarray:
    """
    Returns a subset with high Max-min diversity under partition matroid constraint.
    
    Parameters:
        sets: A tuple containing partitioned sets returned by IFDM function
        k: A decimal integer, the number of elements to be selected
        error: A float number indicating the error parameter
        complete: Complete graph adjacency matrix containing distances between all pairs of points
    
    Returns:
        solution: A numpy array containing selected elements that maximize the minimum pairwise distance
    """
    amount = complete.shape[0]
    complete_array = np.arange(amount)
    U_gmm = GMM(complete_array, k, complete)
    critical_number = len(sets)
    U_c = []
    for i in range(critical_number):
        U_c.append(np.intersect1d(sets[i][0], U_gmm))
    distance_p = 2 * MaxMinDiversity(U_gmm, complete)

    S = []
    while len(S) != 0:
        for c in range(critical_number):
            while len(U_c[c]) < k and any(v for v in sets[c][0] if MaxMinDiversity(np.union1d(U_c[c], [v]), complete) >= distance_p):
                    max_distance = 0
                    max_distance_vector_index = None
                    for i in sets[c][0]:
                        distance = distancePS(U_c[c], i, complete)
                        if distance > max_distance:
                            max_distance = distance
                            max_distance_vector_index = i
                    U_c[c] = np.append(U_c[c], max_distance_vector_index)

        V_p = np.concatenate(U_c)
        E = []
        for i in range(len(V_p)):
             for j in range(i + 1, len(V_p)):
                  if complete[V_p[i]][V_p[j]] < distance_p / 2:
                       E.append((V_p[i], V_p[j]))
        
        color_constraints = dict()
        color_sets = dict()
        for group in range(critical_number):
            color_constraints[group] = sets[group][1]
            color_sets[group] = sets[group][0]
        
        solution = ILP(len(V_p), E, k, color_constraints, color_sets)
        if len(solution) < k:
             distance_p = (1 - error) * distance_p
    
    S = solution
    return S